파이썬으로 실습하는 병렬 컴퓨팅 (2024.04.08.-2024.04.09.)

병렬프로그래밍 예제 실습
===================================================


### 한국과학기술정보연구원 강지훈

***

### 필요 패키지

  - mpi4py
  - numpy
  - random
  - sklearn
  - matplotlib

***


# 1. 벡터와 행렬 연산

## 1.1. 행렬/벡터 만들기

In [2]:
import numpy as np

n = 10

A = np.random.rand(n, n)
B = np.random.rand(n, n)
v = np.random.rand(n)
w = np.random.rand(n)

np.save("A", A)
np.save("B", B)
np.save("v", v)
np.save("w", w)



## 1.2. 벡터 내적

1. 순차코드
   
   <img src = "images/image01.png">

2. 병렬코드 - 등분할

  <img src = "images/image02.png">

In [ ]:
%%writefile v.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    v = np.load("v.npy")
    w = np.load("w.npy")
    n = v.size
else :
    v = None
    w = None
    n = 0

n = comm.bcast(n, root = 0)


##### n_row 크기 정하기 
n_row = # FIX ME

v_row = np.empty(n_row, dtype = np.float64)
w_row = np.empty(n_row, dtype = np.float64)

##### Scatter 함수 호출
comm.Scatter # FIX ME
comm.Scatter # FIX ME

##### 프로세스별 Local sum 
s = np.dot # FIX ME

##### reduce를 이용한 Global sum
s_all = # FIX ME

if rank == 0:
    print(s_all)


In [ ]:
! mpirun -np 2 python v.py

3. 병렬코드 - 비등분할

    <img src = "images/image03.png">

In [ ]:
%%writefile v_var.py
import numpy as np
from mpi4py import MPI

def para_range(n, size, rank) :
    iwork = divmod(n, size) 
    ista = rank * iwork[0] + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1
    return ista, iend

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    v = np.load("v.npy")
    w = np.load("w.npy")
    n = v.size
else :
    v = None
    w = None
    n = 0

n = comm.bcast(n, root = 0)

##### 프로세스별 범위 할당
ista, iend = # FIX ME
n_row =  # FIX ME

n_rows = comm.gather(n_row, root = 0)

v_row = np.empty(n_row, dtype = np.float64)
w_row = np.empty(n_row, dtype = np.float64)

##### Scatter
comm.Scatterv #FIX ME
comm.Scatterv #FIX ME

s = np.dot(v_row,w_row)

##### reduce를 이용한 Global sum
s_all = comm.reduce #FIX ME

if rank == 0:
    print(s_all)


In [ ]:
! mpirun -np 4 python v_var.py

4. para_range 저장

In [ ]:
%%writefile tools.py

def para_range(n, size, rank) :
    iwork = divmod(n, size) 
    ista = rank * iwork[0] + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1
    return ista, iend


## 1.3. 행렬-벡터곱

1. 순차코드
   
    <img src = "images/image04.png">

In [ ]:
A = np.load("A.npy")
v = np.load("v.npy")

b = np.matmul(A,v)
print (b)


2. 행렬의 행 등분할

    <img src = "images/image05.png">

In [ ]:
%%writefile Av.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("A.npy")
    v = np.load("v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)
else :
    A = None
    n = 0
    n = comm.bcast(n, root = 0)
    v = np.empty(n, dtype = np.float64)

n_row = int(n / size)

A_row = np.empty((n_row, n), dtype = np.float64)

##### 행렬의 행 분할
comm.Scatter #FIX ME

comm.Bcast(v, root = 0)

##### 분할된 행렬과의 연산
b = np.matmul #FIX ME

print(b, rank)

In [ ]:
! mpirun -np 2 python Av.py

3. 행렬의 행 비등분할

    <img src = "images/image06.png">

In [ ]:
%%writefile Avar.py

# Matrix A의 Row decomposition

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("A.npy")
    v = np.load("v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)

else :
    A = None
    n = 0
    n = comm.bcast(n, root = 0)
    v = np.empty(n, dtype = np.float64)

ista, iend = para_range(n, size, rank)

n_row = (iend - ista + 1)

A_row = np.empty((n_row, n), dtype = np.float64)

##### 행렬의 행 분할 및 각 프로세스별 크기 지정
n_rows = comm.gather #FIX ME

comm.Scatterv #FIX ME
comm.Bcast(v, root = 0)

b = np.matmul(A_row,v)

print(b, rank)

In [ ]:
! mpirun -np 3 python Avar.py

4. 행렬/벡터의 행 비등분할

    <img src = "images/image07.png">

In [ ]:
%%writefile Av_var.py

# Matrix A의 Row decomposition

from tools import para_range
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("A.npy")
    v = np.load("v.npy")
    n = v.size
    n = comm.bcast(n, root = 0)

else :
    A = None
    v = None
    n = 0
    n = comm.bcast(n, root = 0)

ista, iend = para_range(n, size, rank)

n_row = (iend - ista + 1)

A_row = np.empty((n_row, n), dtype = np.float64)
v_row = np.empty(n_row, dtype = np.float64)

##### 행렬의 행 분할 및 각 프로세스별 크기 지정
n_chunks = comm.gather #FIX ME
n_rows = comm.allgather #FIX ME

comm.Scatterv #FIX ME
comm.Scatterv #FIX ME

##### 분할된 벡터 곱 범위 지정
vsta_list = []
vend_list = []

for i in range(size) :
    vsta_list.append #FIX ME
    vend_list.append #FIX ME

##### Local MV (최초 자신의 벡터부분)
b = np.matmul #FIX ME

##### 송수신 프로세스 지정
inext = #FIX ME
iprev = #FIX ME

for i in range(size - 1) :
    iloc = iprev - i if iprev >= i else iprev - i + size
    v_recv = np.empty(n_rows[iloc], dtype = np.float64)
    ##### 통신
    comm.Sendrecv #FIX ME
    v_row = np.copy(v_recv)
    b += np.matmul(A_row[:,vsta_list[iloc]:vend_list[iloc]], v_row)

print(b, rank)

In [ ]:
! mpirun -np 3 python Av_var.py

## 3. 행렬-행렬 곱

1. 순차코드

    <img src = "images/image08.png">

In [ ]:
A = np.load("A.npy")
B = np.load("B.npy")

C = np.matmul(A, B)
print (C)


2. 행렬A의 행분할

    <img src = "images/image09.png">

In [ ]:
%%writefile AB.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("A.npy")
    B = np.load("B.npy")
    n = A[0].size
    n = comm.bcast(n, root = 0)

else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    B = np.empty((n, n), dtype = np.float64)

n_row = int(n / size)

A_row = np.empty((n_row, n), dtype = np.float64)

##### 행렬 A의 분할
comm.Scatter #FIX ME
comm.Bcast(B, root = 0)

C = np.matmul(A_row,B)

print(C, rank)

In [ ]:
! mpirun -np 2 python AB.py

3. 행렬A의 행분할 (비등분할)

    <img src = "images/image10.png">

In [ ]:
%%writefile AvarB.py

# Matrix A의 Row decomposition

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("A.npy")
    B = np.load("B.npy")
    n = A[0].size
    n = comm.bcast(n, root = 0)
else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    B = np.empty((n, n), dtype = np.float64)

##### 행렬 A의 분할 범위
ista, iend = #FIX ME

n_row = #FIX ME

A_row = np.empty((n_row, n), dtype = np.float64)
n_list = comm.gather(n_row * n, root = 0)

##### 행렬 A의 분할
comm.Scatterv #FIX ME
comm.Bcast(B, root = 0)

C_rows = np.matmul(A_row,B)

print(C_rows, rank)

In [ ]:
! mpirun -np 3 python AvarB.py

4. 행렬A의 행분할, 행렬 B의 열분할

    <img src = "images/image11.png">

In [ ]:
%%writefile ABvar.py

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("A.npy")
    B = np.load("B.npy")
    BT = np.transpose(B).copy()
    n = A[0].size
    n = comm.bcast(n, root = 0)

else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    BT = None

ista, iend = para_range(n, size, rank)

n_row = (iend - ista + 1)

A_row = np.empty((n_row, n), dtype = np.float64)
BT_row = np.empty((n_row, n), dtype = np.float64)
n_rows = comm.allgather(n_row * n)

##### 행렬 A, B의 분할
comm.Scatterv #FIX ME
comm.Scatterv #FIX ME
B_col = np.transpose(BT_row)

inext = rank + 1 if rank < size - 1 else 0
iprev = rank - 1 if rank > 0 else size - 1

C_unordered_rows = np.matmul(A_row, B_col)

for i in range(size - 1) :
    iloc = iprev - i if iprev >= i else iprev - i + size
    B_recv = np.empty(n_rows[iloc], dtype = np.float64)

    ##### 분할된 행렬 B를 송수신하고 A의 분할된 부분과 곱하여 C에 저장
    comm.Sendrecv #FIX ME
    B_col = np.copy(B_recv)
    B_col = #FIX ME
    C_block = #FIX ME
    C_unordered_rows = #FIX ME

print(C_unordered_rows, rank)

In [ ]:
! mpirun -np 3 python ABvar.py

5. 직접 해보기
    - 올바른 크기의 C를 미리 선언하고 적절한 위치에 C_block 을 배치
    - 3의 C_rows와 같은 결과를 얻음
